In [ ]:
# One last time
import numpy as np
import scipy as sp
import pandas as pd

nframe = 2000           # Number of frames (or snapshots)
nat = 195               # Number of atoms
a = 12.55               # Cell size

# Load the twobody data

In [ ]:
twobody = pd.read_hdf('twobody.hdf5', 'twobody')
twobody.head()

# The Pair Correlation Function (or Radial Distribution Function)

Nice picture/description [here](http://www.physics.emory.edu/faculty/weeks//idl/gofr.html)

Basically we want to compute the following for a given pair of atom symbols ($A, B$):

\begin{equation}
    g_{AB}\left(r\right) = \frac{V}{4\pi r^{2}\Delta r MN_{A}N_{B}}\sum_{m=1}^{M}\sum_{a=1}^{N_{A}}\sum_{b=1}^{N_{B}}Q_{m}\left(r_{a}, r_{b}; r, \Delta r\right)
\end{equation}

\begin{equation}
    Q_{m}\left(r_{a}, r_{b}; r, \Delta r\right) = \begin{cases}
    1\ \ if\ r - \frac{\Delta r}{2} \le \left|r_{a} - r_{b}\right|\lt r + \frac{\Delta r}{2}\\
    0\ \ otherwise
    \end{cases}
\end{equation}

Note that that is the analytical form of the equation (meaning continuous values for r). 
As a consequence the denominator is simplified using an approximation for a volume of a 
spherical shell when $\Delta r$ is small.

Note:

\begin{equation}
    \frac{4}{3}\pi\left(r_{i+1}^{3} - r_{i}^{3}\right) \approx 4\pi r_{i}^{2}\Delta r
\end{equation}

Computationally things will be a bit simpler...the summations are simply a histogram and the we don't need to
make the noted approximation above.

Algorithm:

- Select the distances of interest
- Compute the distance histogram
- Multiply by the normalization constant

\begin{equation}
    \frac{V}{4\pi r^{2}\Delta r MN_{A}N_{B}} \equiv \frac{volume}{\left(distance\ count\right)\left(4 / 3 \pi\right)\left(r_{i+1}^{3} - r_{i}^{3}\right)}\ for\ every\ i
\end{equation}

In [ ]:
def pcf(A, B, a, twobody, dr=0.05, start=0.0, end=6.5):
    '''
    Pair correlation function between two atom types.
    '''
    symbols = ''.join(sorted((A, B)))
    distances = twobody.loc[twobody['symbols'] == symbols, 'distance']
    bins = np.arange(start, end, dr)
    bins = np.append(bins, bins[-1] + dr)
    hist, bins = np.histogram(distances, bins)
    n = len(distances)
    m = len(twobody.index.get_level_values('frame').unique())
    vol = a**3
    rho = n / vol
    r = (bins[1:] + bins[:-1]) / 2
    r3 = bins[1:]**3 - bins[:-1]**3
    denom = rho * 4 / 3 * np.pi * r3
    g = hist / denom    
    i = np.cumsum(hist) / m
    return pd.DataFrame.from_dict({'r': r, 'g': g, 'i': i})

# Compute!

In [ ]:
df = pcf('O', 'O', a, twobody)
df.head()

# Plot!

In [ ]:
# Lets modify a copy of the data for plotting
plotdf = df.set_index('r')
plotdf.columns = ['PCF', 'Integration']

In [ ]:
# Generate the plot
ax = plotdf.plot(secondary_y='Integration')
ax.set_ylabel('Pair Correlation Function (PCF: Na, O)')
ax.right_ax.set_ylabel('Integrated PCF')
ax.set_xlabel('Distance ($\AA$)')
patches, labels = ax.get_legend_handles_labels()
patches2, labels2 = ax.right_ax.get_legend_handles_labels()
legend = ax.legend(patches+patches2, labels+labels2, loc='upper center', frameon=True)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('black')

# Save the everything for later

In [ ]:
# Save the figure
fig = ax.get_figure()
fig.savefig('pcf.pdf')

In [ ]:
# Save the pcf data
store = pd.HDFStore('store.hdf5', mode='a')
store.put('NaO', df, format='table', data_columns=True)